<a href="https://colab.research.google.com/github/Bilkouristas/oxf-vis-25/blob/master/VIS_LAB4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: 3D vision

The purpose of this laboratory is to engage with the field of 3D vision through the practical application of 3D reconstruction techniques, using the [Acino dataset](https://github.com/African-Robotics-Unit/AcinoSet). This dataset, which comprises multi-view video of cheetahs in motion, presents a unique opportunity to explore advanced computer vision concepts in a real-world context. The lab's objectives are centered around the analysis of animal motion, with specific emphasis on computing the trajectory and velocity of cheetahs from the recorded videos.

![image](https://camo.githubusercontent.com/4a0f654a81cb2f1fdb233b7e697326de0b1036a80b3beb3e57ca956dfc21aa7f/68747470733a2f2f696d616765732e73717561726573706163652d63646e2e636f6d2f636f6e74656e742f76312f3537663664353163396637343536366635356563663237312f313630383437333235313335352d52364d44324450414758443534314f364b53504f2f6b6531375a77644742546f6464493870446d34386b444a695252696e76796c30696255524a634434326f4d55717378525571716272316d4f4a594b66495052374c6f4451396d58504f6a6f4a6f71793831533249384e5f4e34563176556235416f494949624c5a68565978435257344250753130537433544241555159564b63515268557845545257612d6f713134375474496f4337494959486358534576726d6c426f596d62724b4e5a5f474775696b38746163633450375f645f666e5f302f636865657461685475726e2e706e673f666f726d61743d3235303077)

The dataset you will be working with includes multi-view video footage (`cam2.mp4`, `cam3.mp4`, etc.) and corresponding CSV files (`cam2_fte.csv`, `cam3_fte.csv`, etc.) that contain the frame-by-frame positions of these landmarks. Each row in a CSV file corresponds to a frame in the video, listing the (x, y) coordinates and a likelihood score indicating the confidence of each landmark's detection.

For the this lab, we will give you the starter code for loading the data and plotting the cheetah's joints captured from two different camera views. For a given frame index, we then plot the corresponding points from two videos side by side to observe the motion from different perspectives.

In [ ]:
import gdown
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
gdown.download("https://drive.google.com/uc?id=1DybvFguLonSU6v8vnKcwdoq2UMgBxWNt", "acino_dataset.zip")
!unzip "acino_dataset.zip"

In [ ]:
def load_points_data(csv_file_path):
    """
    Load points and their corresponding frame indices from a CSV file.
    """
    points_df = pd.read_csv(csv_file_path, header=None, skiprows=2)
    frame_indices = points_df.iloc[:, 0].values  # Frame indices
    points_data = points_df.iloc[:, 1:].values  # Points data
    return frame_indices, points_data

def plot_points_on_frame(frame, points):
    """
    Plot the given points on the frame.
    """
    for i in range(0, points.shape[0], 3):  # Iterate through each set of 'x', 'y', 'likelihood'
        x, y, likelihood = points[i], points[i+1], points[i+2]
        # Plot with a red dot if likelihood meets threshold (example: 0.5)
        plt.scatter(x, y, c='red', s=10)

def plot_frame_with_points(video_path, frame_index, points):
    """
    Plot points on the specified frame of a video.
    """
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    success, frame = cap.read()
    cap.release()

    if not success:
        print(f"Failed to read frame at index {frame_index} from {video_path}.")
        return

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame_rgb)
    plot_points_on_frame(frame, points)


def plot_points_for_frame_index(video1_path, csv1_path, video2_path, csv2_path, frame_index):
    """
    Plot points for a specific frame index from two videos.
    """
    frame_indices1, points_data1 = load_points_data(csv1_path)
    frame_indices2, points_data2 = load_points_data(csv2_path)

    index1 = np.where(frame_indices1 == frame_index)[0]
    index2 = np.where(frame_indices2 == frame_index)[0]

    if index1.size == 0 or index2.size == 0:
        print(f"Frame index {frame_index} not found in one of the videos.")
        return

    plt.figure(figsize=(20, 6))

    plt.subplot(1, 2, 1)
    plot_frame_with_points(video1_path, frame_index, points_data1[index1[0]])
    plt.axis('off')
    plt.title(f"Video 1: Frame {frame_index}")

    plt.subplot(1, 2, 2)
    plot_frame_with_points(video2_path, frame_index, points_data2[index2[0]])
    plt.axis('off')
    plt.title(f"Video 2: Frame {frame_index}")

    plt.show()

Now, we can actually plot the cheetah keypoints in the view of camera 2 and camera 3. We start with `frame_index=200` but go ahead and change `frame_index` to some other frames to see how the cheetah and keypoints moves throughout the video.

In [ ]:
# Example usage
video1_path = '/content/acino_dataset/cam2.mp4'
csv1_path = '/content/acino_dataset/cam2_fte.csv'
video2_path = '/content/acino_dataset/cam3.mp4'
csv2_path = '/content/acino_dataset/cam3_fte.csv'
frame_index = 200  # Example frame index

# You should see the same cheetah keypoints from two different camera viewpoints
plot_points_for_frame_index(video1_path, csv1_path, video2_path, csv2_path, frame_index)

Above, we've seen what the keypoints look like when being plotted onto the cheetah. Here, you will need to implement the `get_points_for_frame` to obtain the specified 2D keypoints from the given CSV files and the particular frame_index. You can use `load_points_data` as a helper function.

In [ ]:
def get_points_for_frame(csv1_path, csv2_path, frame_index):
    """
    Returns the points for a specific frame from two CSV files.

    Parameters:
    - csv1_path: Path to the first CSV file.
    - csv2_path: Path to the second CSV file.
    - frame_index: The frame index for which to retrieve the points.

    Returns:
    A tuple containing two numpy arrays: (points_view1, points_view2),
    where each array is the points data for the specified frame from each view.
    If the frame index is not found in one of the CSV files, None is returned for that view.
    """
    # Load the points data using `load_points_data`

    # Find the index of the requested frame in each dataset
    # Hint: use np.where

    # Obtain the actual 2D points located at the specified indices

    return points_view1, points_view2

Having successfully visualized the 2D key points on the cheetah from multiple views, you will now advance to a crucial component of 3D vision: triangulation. Triangulation allows us to infer the three-dimensional position of a point by observing it from two different viewpoints. This process is fundamental in reconstructing the 3D structure of a scene from 2D images.

Your task is to implement a function that triangulates the position of points on the cheetah from two different camera views. This will involve using camera calibration parameters, including intrinsic matrices (**K**), distortion coefficients (**D**), rotation matrices (**R**), and translation vectors (**T**), which are provided in a scene file which are provided in the calibration file of the dataset (`4_cam_scene_sba.json`).

To save you from perusing the OpenCV documentation, we'll provide the following OpenCV function signatures. Familiarize yourself with these OpenCV functions, which will be essential for this task:

- `cv2.undistortPoints`: Corrects radial and tangential distortion in the observed points.

```
def undistortPoints(distorted_pixel, K, distort_coeffs):
"""
Corrects radial and tangential distortion in the observed points.

Params:
    distorted_pts_pixel: (np.ndarray shaped Nx2) The distorted keypoints, expressed in *pixel space*.
    K: (np.ndarray shaped 3x3) The intrinsic matrix.
    distort_coeffs: (np.ndarray shaped 4x1) The distortion coefficients used to correct the radial distortion.

Returns:
    undistorted_pts_screen: (np.ndarray shaped Nx1x2) The undistorted keypoints, expressed in *screen space*.
"""
```

- `cv2.triangulatePoints`: Triangulates the corresponding points from two views, given the projection matrices for each camera.

```
def triangulatePoints(proj_mat1, proj_mat2, undistorted_pts1, undistorted_pts2):
Triangulates the corresponding points from two views, given the projection matrices for each camera.

Params:
    proj_mats1: (np.ndarray shaped 3x4) The projection matrix for camera 1.
    proj_mats2: (np.ndarray shaped 3x4) The projection matrix for camera 2.
    undistorted_pts1: (np.ndarray shaped 2xN) The undistorted keypoints in image 1.
    undistorted_pts2: (np.ndarray shaped 2xN) The undistorted keypoints in image 2.
Returns:
    points_4d_homog: (np.ndarray shaped 4xN) The 4D homogenous keypoints expressed in the world frame.
```


Once you have done this, implement the triangulation using the following steps as a guide:

1. Extract Camera Calibration Parameters: We give the function `load_scene` in the provided `acino_calibration.py` file to load the camera calibration parameters from the scene file. You will then extract the camera extrinsics and intrinsics from this.

2. Prepare Projection Matrices: Construct the projection matrices for each camera view by using the extrinsic parameters (rotation and translation).

3. Correct Distortion: As the images were captured with a fisheye lens, they exhibit significant radial distortion. Apply cv2.undistortPoints to the 2D points from each view, using the respective camera's calibration parameters.

4. Triangulate Points: With the undistorted points and projection matrices, use cv2.triangulatePoints to calculate the 3D coordinates of each point in a homogeneous coordinate system.

5. Convert to 3D Coordinates: Convert the triangulated points from homogeneous coordinates to standard 3D coordinates.

In [ ]:
from acino_dataset.acino_calibration import load_scene
k_arr, d_arr, r_arr, t_arr, camera_res = load_scene('/content/acino_dataset/4_cam_scene_sba.json')

In [ ]:
def triangulatePoints(k_arr, d_arr, r_arr, t_arr, points_view1, points_view2):
    # Extract the Ks, Rs, ts, and distortion coeffs
    # We have 4 cameras in the stero setup but we want camera 2 and 3
    # (Located at indices 1 and 2)
    # Obtain the camera intrinsic matrices from k_arr

    # Obtain the distortion coefficients [k1, k2, p1, p2, k3] from d_arr

    # Obtain the camera extrinsics

    # Compute the entire camera projection extrinsics

    # Obtain the corresponding points in both images, from points_view1 and points_view2, how can we extract 2D x,y coords?

    # Convert points to the shape expected by cv2.undistortPoints (2xN to Nx1x2)

    # Correct radial distortion using cv2.undistortPorts

    # Triangulate points using cv2.triangulatePoints

    # Convert from homogeneous coordinates to 3D

    return points_3d

In [ ]:
frame_index = 200
points_view1, points_view2 = get_points_for_frame(csv1_path, csv2_path, frame_index)
triangulatePoints(k_arr, d_arr, r_arr, t_arr, points_view1, points_view2)

frame_index = 240
points_view1, points_view2 = get_points_for_frame(csv1_path, csv2_path, frame_index)
triangulatePoints(k_arr, d_arr, r_arr, t_arr, points_view1, points_view2)

After successfully triangulating the cheetah's 2D keypoint detections from multiple camera views and converting them to 3D, we are now in a position to analyse the cheetah's motion. This final part of the lab focuses on applying the triangulation process across multiple frames, plotting the resulting 3D trajectory, and computing the velocity of the cheetah over the sequence. This exercise encapsulates the essence of 3D motion analysis.

Triangulate Points Across Multiple Frames: Implement a loop or a function that applies the triangulation process to a sequence of frames. This will involve extracting and triangulating the corresponding points from each frame and accumulating the 3D coordinates.



In [ ]:
def get_trajectory(csv1_path, csv2_path):
    # return the trajectory of the chosen keypoints as a TxNx3 numpy array
    # where T=length of the trajectory, N=number of keypoints


Plotting the 3D Trajectory: Utilize a 3D plotting library (such as matplotlib's 3D plotting capabilities) to visualize the path of the cheetah. Each set of triangulated 3D points should be plotted in sequence, illustrating the trajectory over time.



In [ ]:
def plot_trajectory(trajectory, velocity=None):
    # plot the TxNx3 numpy array as a 3D trajectory
    traj_len = trajectory.shape[0]
    neck_points = trajectory[:,3,:]
    spine_points = trajectory[:,4,:]
    tail_points = trajectory[:,5,:]

    keypoints_dict = {
        "x": np.hstack((neck_points[:, 0], spine_points[:, 0], tail_points[:, 0])),
        "y": np.hstack((neck_points[:, 1], spine_points[:, 1], tail_points[:, 1])),
        "z": np.hstack((neck_points[:, 2], spine_points[:, 2], tail_points[:, 2])),
        "body_part": ["neck"]*traj_len + ["spine"]*traj_len + ["tail"]*traj_len,
        "index": np.tile(np.arange(traj_len), 3)

    }
    if velocity is not None:
        velocity = np.concatenate((velocity[0][None,:,:], velocity), axis=0)
        neck_vel = velocity[:,3,:]
        spine_vel = velocity[:,4,:]
        tail_vel = velocity[:,5,:]
        keypoints_dict["velocity"] = np.hstack((neck_vel[:, 0], spine_vel[:, 0], tail_vel[:, 0]))

    keypoints_df = pd.DataFrame(keypoints_dict)
    fig = go.Figure()
    if velocity is not None:
        fig = px.scatter_3d(keypoints_df, x="x", y="y", z="z", color="velocity",
                            hover_name="body_part", color_continuous_scale="agsunset",
                            range_x=[0, 12], range_y=[0, 12], range_z=[0, 1])
    else:
        fig = px.scatter_3d(keypoints_df, x="x", y="y", z="z", color="index",
                            hover_name="body_part", color_continuous_scale="agsunset",
                            range_x=[0, 12], range_y=[0, 12], range_z=[0, 1])
    fig.update_traces(marker=dict(size=3))
    fig.show()

plot_trajectory(trajectory)

In [ ]:
fig = go.Figure()

num_keypoints = 20
num_frames = 200

# Add initial state (first frame)
for i in range(num_keypoints):
    fig.add_trace(go.Scatter3d(x=[trajectory[0, i, 0]],
                               y=[trajectory[0, i, 1]],
                               z=[trajectory[0, i, 2]],
                               mode='markers',
                               name=f'Keypoint {i+1}'))

# Create frames for each time step
frames = [go.Frame(data=[go.Scatter3d(x=trajectory[frame, :, 0],
                                      y=trajectory[frame, :, 1],
                                      z=trajectory[frame, :, 2],
                                      mode='markers',
                                      marker=dict(size=5, symbol='circle'))
                         for _ in range(num_keypoints)])
          for frame in range(1, num_frames)]

fig.frames = frames

# Layout settings
fig.update_layout(updatemenus=[dict(type="buttons",
                                    buttons=[dict(label="Play",
                                                  method="animate",
                                                  args=[None, {"frame": {"duration": 50}, "fromcurrent": True}]),
                                             dict(label="Pause",
                                                  method="animate",
                                                  args=[[None], {"frame": {"duration": 0}, "mode": "immediate"}])])],
                  title='Keypoint Trajectory Animation')

# Show the figure
fig.show()

Velocity Computation: Calculate the cheetah's velocity between consecutive frames. This can be achieved by computing the distance between points in successive frames and dividing by the time interval between those frames. Consider the frame rate of the video to determine the time interval.

In [ ]:
def calculate_velocity():
  # plot the TxNx3 numpy array as a 3D trajectory

In [ ]:
plot_trajectory(trajectory, velocity=vel)

## Bonus tasks:

**Data Smoothing**: The raw triangulation data may contain noise or inaccuracies due to measurement errors or limitations in the detection algorithm. Try to apply temporal smoothing or filtering techniques to the 3D trajectory data to achieve more accurate velocity calculations.

**Visualization Enhancements**: Enhance your 3D trajectory plot with features such as color gradients to indicate velocity